# Notebook for Creating High Wage Outputs

#### This notebook is currently written to create high wage outputs for 2020.

In [1]:
import pandas as pd
import numpy as np
import string
import warnings
import os
import re
from jqi_functions import *
warnings.filterwarnings('ignore')

## Ensure that IPUMS data is in the proper file location

The desired year for IPUMS data should live in the data folder, under `data/ipums` with the naming convention as `IPUMS_{YEAR}.csv`, where `{YEAR}` should match the year entered below.

A full example for the file path would be `data/ipums/IPUMS_2020.csv`.

## Set the desired year and the corresponding cost of living year

In [2]:
year = '2010'

## Creating IPUMS dataframe

#### IPUMS Data
`cleaned_ipums` is a function to generate a cleaned pandas dataframe using IPUMS data, filtering it down to California only and the desired year. The year needs to be entered in string format as a parameter.

In [3]:
ca_ipums = cleaned_ipums(year)

#### Cost of living needs to be updated each year.

In [4]:
cost_of_living = pd.read_csv(f'data/cost_of_living/united-way-col-1A1PS1C{year}.csv')

### Create county lookup dataframe

Expanding the `county_info` dataframe to include cost of living metrics. This dataframe is used when industry information in a geographic area is too sparse and the next largest geographic area needs to be used instead.

In [5]:
county_info = pd.read_csv('data/county_to_regions_key.csv')

In [6]:
county_info = county_info[['County', 'COUNTYFIP', 'Rural/Urban', 'CERF Regions', 'Population']]

In [7]:
county_info = pd.merge(county_info, cost_of_living, left_on = 'CERF Regions', right_on = 'Regions')

In [8]:
county_info = county_info.rename(columns = {'Cost of Living':'Regional COL'})
county_info = county_info.drop(columns=['Regions'])

In [9]:
county_info['State COL'] = cost_of_living.iloc[13][1]

View of final `county_info` dataframe.

In [10]:
county_info.head()

,County,COUNTYFIP,Rural/Urban,CERF Regions,Population,Regional COL,State COL
0,Alameda,1,Urban,Bay Area,1656754,82928.33,66134.41
1,Contra Costa,13,Urban,Bay Area,1142251,82928.33,66134.41
2,Solano,95,Urban,Bay Area,441829,82928.33,66134.41
3,San Mateo,81,Urban,Bay Area,767423,82928.33,66134.41
4,Santa Clara,85,Urban,Bay Area,1927470,82928.33,66134.41


In [11]:
ca_ipums = pd.merge(ca_ipums, county_info, on = 'COUNTYFIP')

View of final `ca_ipums` dataframe.

In [12]:
ca_ipums.head()

,YEAR,COUNTYFIP,INDNAICS,PERWT,INCWAGE,NAICS Code,Industry Title,Industry,Crosswalk Value,County,Rural/Urban,CERF Regions,Population,Regional COL,State COL
0,2010,75,3341,76.0,195000,3341,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,8,San Francisco,Urban,Bay Area,874961,82928.33,66134.41
1,2010,75,3341,76.0,195000,3341,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,9,San Francisco,Urban,Bay Area,874961,82928.33,66134.41
2,2010,75,3341,58.0,25000,3341,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,8,San Francisco,Urban,Bay Area,874961,82928.33,66134.41
3,2010,75,3341,58.0,25000,3341,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,9,San Francisco,Urban,Bay Area,874961,82928.33,66134.41
4,2010,75,3341,54.0,140000,3341,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,8,San Francisco,Urban,Bay Area,874961,82928.33,66134.41


## Create EDD Dataframe

#### EDD Data
The year for EDD data must be specified.

These CSV files are filtered and cleaned versions of the raw EDD Current Employment Statistics dataset. These CSV files can be created for upcoming years with the notebook `multiyear-edd-data-creation.ipynb`

In [13]:
edd = pd.read_csv(f'data/edd/edd_{year}.csv')

View of final `edd` dataframe.

In [14]:
edd.head()

,Area Type,Area Name,Year,Month,Date,Series Code,Seasonally Adjusted,Current Employment,Industry Title,COUNTYFIP,County,Rural/Urban,CERF Regions,Crosswalk Value
0,County,Alameda,2010,January,01/01/2010,80000000,N,22300,other services,1,Alameda,Urban,Bay Area,32
1,County,Alameda,2010,February,02/01/2010,80000000,N,22500,other services,1,Alameda,Urban,Bay Area,32
2,County,Alameda,2010,March,03/01/2010,80000000,N,22900,other services,1,Alameda,Urban,Bay Area,32
3,County,Alameda,2010,April,04/01/2010,80000000,N,23200,other services,1,Alameda,Urban,Bay Area,32
4,County,Alameda,2010,May,05/01/2010,80000000,N,23400,other services,1,Alameda,Urban,Bay Area,32


## Add High Wage Features

`add_geo_high_wages` is a function that adds the following engineered features:
- Above Threshold (Number of records above respective cost of living threshold)
- Weighted above threshold (Above Threshold multiplied by person weight variable)
- Unweighted industry counts (Number of records in that industry)
- Weighted industry counts (Sum of person weight values in that industry)
- Weighted high wage percentage (Weighted Above Threshold divided by Weighted Industry Counts as a percentage)

The features are created for the following geographical levels:
- Region
- California

In [15]:
ca_ipums_hw = add_geo_high_wages(ca_ipums)

View of final `ca_ipums_hw` dataframe.

In [16]:
ca_ipums_hw.head().T

,0,1,2,3,4
YEAR,2010,2010,2010,2010,2010
COUNTYFIP,75,75,75,75,75
INDNAICS,3341,3341,3341,3341,3341
PERWT,76.0,58.0,54.0,56.0,110.0
INCWAGE,195000,25000,140000,125000,200000
NAICS Code,3341,3341,3341,3341,3341
Industry Title,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing
Industry,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing
Crosswalk Value,8,8,8,8,8
County,San Francisco,San Francisco,San Francisco,San Francisco,San Francisco


## Create High Wage Outputs Dataframe

`edd_to_hw` is the function that outputs the values needed to create the high wage output dataframe. This portion of the notebook runs through every unique combination of region, industry, and date, to get that respective output and add it to the dataframe.

Because of the nested structure of the EDD industries, only a small selection of EDD industries can be used to ensure that individuals in nested industries are not counted twice. The selection of these industries different per region, so the series code of each industry is documented in the `region_series_codes` global variable in the `jqi_functions.py` library. Each of these codes were then assigned their own crosswalk value, which align with each crosswalk value assigned to each IPUMS industry. For generating high wage outputs, we only iterate through the EDD industries that have been selected and have a designated crosswalk value.

Getting unique values for each region, industry, and date.

In [17]:
regions_ipums = ca_ipums_hw['CERF Regions'].unique()

In [18]:
crosswalk_vals = sorted(edd['Crosswalk Value'].unique())

In [19]:
dates_edd = edd['Date'].unique()

Initializing empty lists for the function's outputs to later be joined in a dataframe.

In [20]:
industries = []
dates = []
regions = []
counts = []
emp_counts = []

In [21]:
total_iterations = len(regions_ipums) * len(crosswalk_vals) * len(dates_edd)

For loop to populate lists for the high wage outputs. This will take some time to finish running.

In [22]:
progress_count = 0
for region in regions_ipums:
    for code in crosswalk_vals:
        for date in dates_edd:
            hw_count, hw_perc, employment_count, industry = edd_to_hw(edd, ca_ipums_hw, region, code, date, 10)
            industries.append(industry)
            dates.append(date)
            regions.append(region)
            counts.append(hw_count)
            emp_counts.append(employment_count)
            progress_count += 1
            if progress_count % (total_iterations / 10) == 0:
                percent_done = int((progress_count / total_iterations) * 100)
                print(f'Progress: {percent_done}% Complete')

Progress: 10% Complete
Progress: 20% Complete
Progress: 30% Complete
Progress: 40% Complete
Progress: 50% Complete
Progress: 60% Complete
Progress: 70% Complete
Progress: 80% Complete
Progress: 90% Complete
Progress: 100% Complete


Creating a cleaned dataframe from the output lists.

In [23]:
df_dict = {'Industry':industries, 'Date':dates, 'Region':regions, 'High Wage Count':counts, 'Employment Count':emp_counts}
hw_output = pd.DataFrame(df_dict)
hw_output = hw_output[hw_output['Industry'].notna()]
hw_output['Date']= pd.to_datetime(hw_output['Date'])
hw_output['High Wage Count'] = hw_output['High Wage Count'].astype(int)
hw_output = hw_output.sort_values(by=['Industry', 'Region', 'Date'])
hw_output = pd.merge(hw_output, cost_of_living, left_on='Region', right_on='Regions')
hw_output = hw_output[['Industry', 'Date', 'Region', 'High Wage Count', 'Employment Count', 'Cost of Living']]
hw_output = hw_output.drop_duplicates()

Add Region Population to Dataframe

In [24]:
reg_pop = county_info.groupby(by='CERF Regions').sum()[['Population']].reset_index()
reg_pop

,CERF Regions,Population
0,Bay Area,7710026
1,Central Coast,2342005
2,Central San Joaquin,1752543
3,Eastern Sierra,188734
4,Inland Empire,4560470
5,Kern,887641
6,Los Angeles,10081570
7,North State,713754
8,Northern San Joaquin,1557179
9,Orange,3168044


In [25]:
hw_output = pd.merge(hw_output, reg_pop, left_on='Region', right_on='CERF Regions')
hw_output = hw_output.drop(columns=['CERF Regions'])
hw_output = hw_output.rename(columns={"Population": "Region Population"})

View of final `hw_output` dataframe.

In [26]:
hw_output.head()

,Industry,Date,Region,High Wage Count,Employment Count,Cost of Living,Region Population
0,accommodation and food services,2010-01-01,Inland Empire,9598,105200.0,53849.84,4560470
1,accommodation and food services,2010-02-01,Inland Empire,9644,105700.0,53849.84,4560470
2,accommodation and food services,2010-03-01,Inland Empire,9735,106700.0,53849.84,4560470
3,accommodation and food services,2010-04-01,Inland Empire,9890,108400.0,53849.84,4560470
4,accommodation and food services,2010-05-01,Inland Empire,9890,108400.0,53849.84,4560470


Code to export the dataframe as a CSV file - change file path if needed and uncomment to run.

In [27]:
hw_output.to_csv(f'data/outputs/hw_outputs_{year}.csv', encoding='utf-8', index=False)

#### TESTING

In [28]:
bayarea = hw_output.loc[hw_output['Date'] == f'{year}-12-01'].loc[hw_output['Region'] == 'Bay Area']

In [29]:
bayarea['High Wage Count'].sum()

704734

In [30]:
bayarea['Employment Count'].sum()

3007700.0

In [31]:
california = hw_output.loc[hw_output['Date'] == f'{year}-12-01']

In [32]:
california['High Wage Count'].sum()

3412297

In [33]:
california['Employment Count'].sum()

13892900.0

https://data.bls.gov/timeseries/LASST060000000000005?amp%253bdata_tool=XGtable&output_view=data&include_graphs=true

## Code for concatenating multiple years (2010-2020)

In [38]:
# hw_2010 = pd.read_csv('data/outputs/hw_outputs_2010.csv')
# hw_2011 = pd.read_csv('data/outputs/hw_outputs_2011.csv')
# hw_2012 = pd.read_csv('data/outputs/hw_outputs_2012.csv')
# hw_2013 = pd.read_csv('data/outputs/hw_outputs_2013.csv')
# hw_2014 = pd.read_csv('data/outputs/hw_outputs_2014.csv')
# hw_2015 = pd.read_csv('data/outputs/hw_outputs_2015.csv')
# hw_2016 = pd.read_csv('data/outputs/hw_outputs_2016.csv')
# hw_2017 = pd.read_csv('data/outputs/hw_outputs_2017.csv')
# hw_2018 = pd.read_csv('data/outputs/hw_outputs_2018.csv')
# hw_2019 = pd.read_csv('data/outputs/hw_outputs_2019.csv')
# hw_2020 = pd.read_csv('data/outputs/hw_outputs_2020.csv')

In [39]:
# hw_2010['Year'] = 2010
# hw_2011['Year'] = 2011
# hw_2012['Year'] = 2012
# hw_2013['Year'] = 2013
# hw_2014['Year'] = 2014
# hw_2015['Year'] = 2015
# hw_2016['Year'] = 2016
# hw_2017['Year'] = 2017
# hw_2018['Year'] = 2018
# hw_2019['Year'] = 2019
# hw_2020['Year'] = 2020

In [40]:
# hw_output_concat = pd.concat([hw_2010, hw_2011, hw_2012, hw_2013, 
#                               hw_2014, hw_2015, hw_2016, hw_2017, 
#                               hw_2018, hw_2019, hw_2020])

In [41]:
# hw_output_concat.to_csv('data/outputs/hw_outputs_multiyear.csv', encoding='utf-8', index=False)